## Cython/Python

Here we will program the above cells in Python and call a Monte-Carlo function to yield us the outputs we desire. The Monte-Carlo function will be written in Cython and we will use the %timeit cell magic to give us a run-time of our Cython function. 

### Cython Function

In [ ]:
%load_ext cython

In [ ]:
%%cython -lgsl -lgslcblas

#!python
#cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True

#Import modules: 

#from cython.parallel import prange, parallel
from libc.stdlib cimport rand, RAND_MAX, calloc, malloc, realloc, free, abort
from libc.math cimport log, exp

#Use the CythonGSL package to get the low-level routines
from cython_gsl cimport *

######################### Define the Data Structure ############################

cdef struct Parameters:
    #Pointer for Y data array
    double* Y
    #size of the array
    int* Size

################ Support Functions for Monte-Carlo Function ##################

#Create a function that allocates the memory and verifies integrity
cdef int alloc_struct(Parameters* data, int* N, unsigned int flag) nogil:
    
    cdef int Mem_Int = True
    
    #fill in the size of the array
    data.Size = N
    
    #allocate the data array initially
    if flag==0:
        data.Y = <double*> calloc(N[0], sizeof(double))
    #reallocate the data array
    else:
        data.Y = <double*> realloc(data.Y, N[0] * sizeof(double))
    
    #If the elements of the struct are not properly allocated, destory it and return null
    if N[0]!=0 and data.Y==NULL:
        
        #return the memory to system
        destroy_struct(data)
        
        #update the memory integrity variable to False
        Mem_Int = False
    
    return Mem_Int

#Create the destructor of the struct to return memory to system
cdef void destroy_struct(Parameters* data) nogil:
    free(data.Y)
    free(data)

#This function fills in the Y observed variable with discreet 0/1
cdef void Y_fill(Parameters* data, double p_true):
    
    cdef:
        Py_ssize_t i
        double y
    
    for i in range(data.Size[0]):
        
        y = rand()/<double>RAND_MAX
        
        if y <= p_true:
            data.Y[i] = 1.0

#Definition of the function to be maximized: LLF of Bernoulli
cdef double LLF(double p, void* data) nogil:
    
    cdef:
        #the sample structure (considered the parameter here), recast
        Parameters* sample = <Parameters*> data
        
        #the loop iterator
        Py_ssize_t i, 
        int n = sample.Size[0]
        
        #the total of the LLF
        double Sum = 0
    
    for i in range(n):
        
        if sample.Y[1]==1.0: Sum += log(p)
        else: Sum += log(1-p)
    
    return (-Sum/n)

########################## Monte-Carlo Function ##############################

def Monte_Carlo(int[::1] Samples, double[:,::1] p_hat, 
                Py_ssize_t Sims, double p_true):
     
    #Define variables and pointers
    cdef:
        #Data Structure
        Parameters* Data
            
        #iterators
        Py_ssize_t i, j
        int status, GSL_CONTINUE, Iter = 0, max_Iter = 100 
        
        #Variables
        int N = Samples.shape[0], Mem_Int
        double a, b, tol = 1e-6, start_val
        
        #GSL objects and pointer
        const gsl_min_fminimizer_type* T
        gsl_min_fminimizer* s
        gsl_function F
    
    #Allocate the minimization routine
    T = gsl_min_fminimizer_brent
    s = gsl_min_fminimizer_alloc(T)
    
    #allocate the struct dynamically
    Data = <Parameters*> malloc(sizeof(Parameters))
    
    #verify memory integrity
    if Data==NULL or s==NULL: abort()
        
    #Set the GSL function
    F.function = &LLF
    F.params = <void*> Data
    
    try:
        
        for i in range(N): 

            #allocate the elements of the struct (if i>0, reallocate data array)
            Mem_Int = alloc_struct(Data, &Samples[i], i)

            #verify memory integrity
            if Mem_Int==False: abort() 

            for j in range(Sims):

                #fill the array in the struct
                Y_fill(Data, p_true)

                #set the parameters for the GSL function (the samples)
                a = tol
                b = 1000
                
                #set the starting value (random number)
                start_val = rand()/<double>RAND_MAX
               
                #set the minimizer
                gsl_min_fminimizer_set(s, &F, start_val, a, b)  

                #initialize conditions
                GSL_CONTINUE = -2
                status = -2
                Iter = 0

                while (status == GSL_CONTINUE and Iter < max_Iter):

                    Iter += 1
                    status = gsl_min_fminimizer_iterate(s)

                    start_val = gsl_min_fminimizer_x_minimum(s)
                    a = gsl_min_fminimizer_x_lower(s)
                    b = gsl_min_fminimizer_x_upper(s)

                    status = gsl_min_test_interval(a, b, tol, 0.0)

                    if (status == GSL_SUCCESS):
                        p_hat[i,j] = start_val

    finally:
        destroy_struct(Data)
        gsl_min_fminimizer_free(s)

### Python Portion and Cython Function Call

In [ ]:
import numpy as np

In [ ]:
#First we will recreate the first cell in the matlab example in python

#Sample Sizes
N = np.array([5,50,500,5000], dtype='i')

#Parameters for MC
T = 1000
p_true = 0.2

#Array of the outputs from the MC
p_hat = np.empty((N.size,T), dtype='d')
p_hat.fill(np.nan)

In [ ]:
Monte_Carlo(N, p_hat, T, p_true)